In [4]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import h5py
import keras.backend as K
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Activation
from keras import optimizers, layers, callbacks
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

## Intangible improvement (IM6, CB8,OS3,IM4,IM5,OS4)

In [5]:
x_1 = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Separated dataset_ML/a1_traintest_py.csv', 
                    delimiter = ',', usecols = (25,38,18,23,24,19), skip_header=1)
y_1o = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Separated dataset_ML/a1_traintest_py.csv', 
                   delimiter =',', usecols = (65), skip_header=1).reshape (54,1)
x_2 = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Separated dataset_ML/a1_cv_py.csv', 
                    delimiter = ',', usecols = (25,38,18,23,24,19), skip_header=1)
y_2o = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Separated dataset_ML/a1_cv_py.csv',
                      delimiter =',', usecols = (65),skip_header=1).reshape (14,1)

In [6]:
enc = OneHotEncoder ()
enc.fit (y_1o)
y_train = enc.transform(y_1o).toarray()
y_test = enc.transform(y_2o).toarray()

In [7]:
scaler = StandardScaler ()
scaler.fit (x_1)
x_train = scaler.transform (x_1)
x_train, x_cv, y_train, y_cv = train_test_split (x_train,y_train,test_size=0.13)
x_test = scaler.transform (x_2)

In [9]:
num_classes = 4;

model = Sequential ()
model.add (Dense (units = 25, input_dim = 6))
model.add (layers.Activation ('tanh'))
model.add (Dense (units = 25))
model.add (layers.Activation ('tanh'))
model.add (Dense (units = 25))
model.add (layers.Activation ('tanh'))
#model.add (Dense (units = 16))
model.add (Dense(num_classes, activation = 'softmax'))

#sgd = optimizer.SGD (lr=0.01, momentum = 0, decay =0, nestrov = False)
model.compile (loss = 'binary_crossentropy', optimizer = 'sgd', 
               metrics=['binary_accuracy'])

History = model.fit(x_train, y_train,
                    batch_size=60,
                    epochs=40,
                    verbose=1, validation_data=(x_cv, y_cv))

print(model.evaluate (x_test, y_test, batch_size = 60, verbose =1))

#help (model.evaluate)
#this is the output units

y_pred_train = model.predict (x_train)
y_pred_cv = model.predict (x_cv)
y_pred_test = model.predict (x_test)
y_pred_whole = np.vstack ((y_pred_train, y_pred_cv, y_pred_test)).astype(np.float32)

Train on 46 samples, validate on 8 samples
Epoch 1/40
46/46 [==============================] - ETA: 0s - loss: 0.5996 - binary_accuracy: 0.7337 - val_loss: 0.6186 - val_binary_accuracy: 0.6875
Epoch 2/40
46/46 [==============================] - ETA: 0s - loss: 0.5984 - binary_accuracy: 0.7337 - val_loss: 0.6168 - val_binary_accuracy: 0.6875
Epoch 3/40
46/46 [==============================] - ETA: 0s - loss: 0.5972 - binary_accuracy: 0.7337 - val_loss: 0.6151 - val_binary_accuracy: 0.7188
Epoch 4/40
46/46 [==============================] - ETA: 0s - loss: 0.5960 - binary_accuracy: 0.7337 - val_loss: 0.6135 - val_binary_accuracy: 0.7188
Epoch 5/40
46/46 [==============================] - ETA: 0s - loss: 0.5949 - binary_accuracy: 0.7337 - val_loss: 0.6118 - val_binary_accuracy: 0.7188
Epoch 6/40
46/46 [==============================] - ETA: 0s - loss: 0.5938 - binary_accuracy: 0.7337 - val_loss: 0.6102 - val_binary_accuracy: 0.7188
Epoch 7/40
46/46 [==============================] - ETA: 

### t(x) for labelling

In [10]:
#save model to json, weights to HDF5
model_json = model.to_json ()
with open ('model_lod_intang_improve_a.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('model_lod_intang_improve_a.h5')
print ('Saved model to disk')

Saved model to disk


In [11]:
#load json and create model
json_file = open('model_lod_intang_improve_a.json', 'r')
loaded_model_json = json_file.read ()
json_file.close ()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights('model_lod_intang_improve_a.h5')

In [12]:
loaded_model.compile(loss = 'binary_crossentropy', optimizer = 'sgd', 
               metrics=['binary_accuracy'])
print(loaded_model.evaluate (x_test, y_test, batch_size = 20, verbose =1))
y_pred_test = loaded_model.predict (x_test)
print(y_pred_test)


i = 0;
j = 0;
m = np.zeros((14,4));
for i in range (14):
    max = 0.0000001;
    for j in range (4):
        if y_pred_test.item ((i,j))>=max:
            max = y_pred_test.item ((i,j))
    for j in range (4):
        if y_pred_test.item ((i,j)) ==max:
            np.put(m, [4*i+j], [1])

i = 0;
j = 0;
selected_LOD = np.zeros((14,1));
for i in range (14):
    for j in range (4):
        if m.item((i,j))>0:
            selected_LOD_1 = m.item((i,j))*y_pred_test.item((i,j));
            np.put(selected_LOD, [i], [selected_LOD_1])

i = 0;
j = 0;
cf = np.zeros((14,4))
for i in range (14):
    for j in range (4):
        cf_1 = y_pred_test.item((i,j))-selected_LOD.item((i,0));
        cf_2 = 10*cf_1;
        np.put(cf, [4*i+j], [cf_2])
print (cf)
np.savetxt('y_cost_LODintang_improve.csv', cf, delimiter=',') 

14/14 [==============================] - ETA: 0s
[0.58797603845596313, 0.7321428656578064]
[[ 0.25270429  0.30054247  0.24075301  0.20600027]
 [ 0.23971331  0.36091769  0.21748932  0.18187971]
 [ 0.61223918  0.09068605  0.1545907   0.14248402]
 [ 0.17407142  0.18672027  0.34739488  0.2918134 ]
 [ 0.11045768  0.23836249  0.37627441  0.27490544]
 [ 0.28150579  0.28275043  0.22472489  0.21101883]
 [ 0.2624343   0.22082227  0.25887683  0.25786659]
 [ 0.21612778  0.36057714  0.2239989   0.19929616]
 [ 0.47204304  0.22568345  0.17846686  0.12380667]
 [ 0.27568361  0.36351267  0.18194067  0.17886306]
 [ 0.16537012  0.31121409  0.23299307  0.29042271]
 [ 0.05760452  0.30553383  0.23661119  0.40025052]
 [ 0.30253083  0.16091831  0.27868772  0.25786316]
 [ 0.15130186  0.21743488  0.2392959   0.39196736]]
[[-0.47838181  0.         -0.59789464 -0.94542205]
 [-1.21204376  0.         -1.4342837  -1.79037973]
 [ 0.         -5.21553129 -4.57648486 -4.69755158]
 [-1.73323467 -1.60674617  0.         -0.